In [1]:
import dqc
import torch
import dqc.utils

In [10]:
dtype = torch.float64
atomposs = torch.tensor([[-0.5, 0.0, 0.0], [0.5, 0.0, 0.0]], dtype=dtype)
basis = "pc-1"

In [11]:
# rough implementation of forcing ground state
def eval_state(mol, force_gs=False):
    qc = dqc.HF(mol).run(dm0=None if not force_gs else "1e")
    ene = qc.energy()
    eival = dqc.lowest_eival_orb_hessian(qc)
    mult = 1e-3
    while eival < -1e-3 and force_gs:
        dm0 = qc.aodm()
        dm0 = dqc.utils.SpinParam.apply_fcn(lambda dm0: dm0 + torch.randn_like(dm0) * mult, dm0)
        dm0 = dqc.utils.SpinParam.apply_fcn(lambda dm0: (dm0 + dm0.transpose(-2, -1).conj()) * 0.5, dm0)
        qc = dqc.HF(mol).run(dm0=dm0)
        ene = qc.energy()
        eival = dqc.lowest_eival_orb_hessian(qc)
        mult *= 2
    return ene, eival

In [12]:
# O2
mol = dqc.Mol(moldesc=([8, 8], atomposs * 2.0), basis=basis, dtype=dtype, spin=2)
print(eval_state(mol, force_gs=False))

(tensor(-149.1737, dtype=torch.float64), tensor([-0.7331], dtype=torch.float64, grad_fn=<SliceBackward>))


In [13]:
print(eval_state(mol, force_gs=True))

(tensor(-149.5416, dtype=torch.float64), tensor([-6.1734e-14], dtype=torch.float64, grad_fn=<SliceBackward>))


In [14]:
# BeH
mol = dqc.Mol(moldesc=([4, 1], atomposs * 2.5), basis=basis, dtype=dtype, spin=1)
print(eval_state(mol, force_gs=False))

(tensor(-15.0064, dtype=torch.float64), tensor([-0.2485], dtype=torch.float64, grad_fn=<SliceBackward>))


In [15]:
print(eval_state(mol, force_gs=True))

(tensor(-15.1374, dtype=torch.float64), tensor([-4.1416e-14], dtype=torch.float64, grad_fn=<SliceBackward>))


In [16]:
# CH
mol = dqc.Mol(moldesc=([6, 1], atomposs * 2.0), basis=basis, dtype=dtype, spin=1)
print(eval_state(mol, force_gs=False))

(tensor(-38.2563, dtype=torch.float64), tensor([-0.0672], dtype=torch.float64, grad_fn=<SliceBackward>))


In [17]:
print(eval_state(mol, force_gs=True))

(tensor(-38.2593, dtype=torch.float64), tensor([-4.8197e-08], dtype=torch.float64, grad_fn=<SliceBackward>))
